bash
`python -m vllm.entrypoints.openai.api_server --model ./merge  --served-model-name Qwen2-7B-Instruct-lora --max-model-len=4096 --enforce-eager`

In [1]:
import json
import os
from pprint import pprint
import re
from tqdm import tqdm
import random

import uuid
import openai
from openai import OpenAI
import tiktoken
import json
import numpy as np
import requests
from scipy import sparse
#from rank_bm25 import BM25Okapi
#import jieba
from http import HTTPStatus


from concurrent.futures import ThreadPoolExecutor, as_completed
from loguru import logger
import json
import time
from tqdm import tqdm



In [2]:
def get_prompt(instruction, input):
    # 提取选项部分
    options = []
    for line in input.split('; '):
        if line.startswith('A:') or line.startswith('B:') or line.startswith('C:') or line.startswith('D:'):
            options.append(line.split(':', 1)[1])

    # 构建选项字符串
    options_str = '\n'.join(f"{'ABCD'[i]}. {o}" for i, o in enumerate(options))
    
    # 构建prompt
    prompt = f"""### Context:
你是一位逻辑推理专家，擅长解决各种逻辑推理问题。

### Observation:
以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都基于闭世界假设，即未观测事实都为假。

### Skills:
你需要运用你的逻辑推理能力，仔细分析并得出正确答案。

### Task:
请逐步分析问题并得出结论。

### Action:
根据你的分析，只返回最终的答案。确保回复的格式是 "A" "B" "C" "D" 四个选项中的一个。不要提供任何解释或多余信息。如果无法推理出结果或问题模糊不清，请直接返回 "A"。

### Result:
最终答案应该只包括一个字母选项，确保回答简洁明了。

### example  
提问：有几种不同的方法来计算数字的幂。在这里，你需要根据不同的计算步骤确定相应的结果。选择题 1：若数 2 将自己乘 3 次，而起始乘积因数为 7，那么最终结果是多少？ A: 14; B: 32; C: 56; D: 64  
解题思路：题目中提到“数 2 将与自己乘 3 次”，这意味着我们要计算 2^3。同时，题目还提到“起始乘积因数为 7”，这意味着我们需要将计算得到的 2^3 的结果乘以 7。
按照这个逻辑，我们先计算 2^3：
2^3=2×2×2=8
然后，我们将这个结果乘以起始乘积因数 7： 
8×7=56
因此，最终结果是 56。所以正确答案是 C 。

### 题目:
{instruction}

### 问题:
{input}
"""

    return prompt
    
def read_and_process_json_file(file_path):
    prompts = []

    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for item in data:
        instruction = item["instruction"]
        input = item["input"]
        prompt = get_prompt(instruction, input)
        prompts.append(prompt)

    return prompts

# prompts = read_and_process_json_file(input_file_path) ##list

In [3]:
def call_qwen_api(MODEL_NAME, prompt):
    # 这里采用dashscope的api调用模型推理，通过http传输的json封装返回结果
    openai_api_key = "EMPTY"
    openai_api_base = "http://localhost:8000/v1"
    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )
    completion = client.chat.completions.create(
      model=MODEL_NAME,
      messages=[
                # {'role':'system','content':'你是一个解决推理任务的专家，你需要分析出问题中的每个实体以及响应关系。然后根据问题一步步推理出结果。并且给出正确的结论。'},

        {"role": "user", "content": prompt}
      ]
    )
    return completion.choices[0].message.content

In [4]:
def most_frequent_char(char1, char2, char3,char4,char5):
    # 创建一个字典来存储每个字符的出现次数
    frequency = {char1: 0, char2: 0, char3: 0,char4: 0,char5: 0}
    
    # 增加每个字符的出现次数
    frequency[char1] += 1
    frequency[char2] += 1
    frequency[char3] += 1
    frequency[char4] += 1
    frequency[char5] += 1
    
    # 找到出现次数最多的字符
    most_frequent = max(frequency, key=frequency.get)
    
    return most_frequent


In [5]:
def get_answer(prompts,MODEL_NAME):
    answer_list = []

    # 送入多线程任务
    for prompt in tqdm(prompts, desc="Submitting tasks", total=len(prompts)):
        # 统一使用llm 三次调用
        # res,res1,res2 = call_qwen_api(MODEL_NAME, prompt),call_qwen_api(MODEL_NAME, prompt),call_qwen_api(MODEL_NAME, prompt)
        # # 通过投票函数获取最终结果并返回
        # ans = most_frequent_char(res,res1,res2)
        res,res1,res2,res3,res4 = call_qwen_api(MODEL_NAME, prompt),call_qwen_api(MODEL_NAME, prompt),call_qwen_api(MODEL_NAME, prompt),call_qwen_api(MODEL_NAME, prompt),call_qwen_api(MODEL_NAME, prompt)
        # 通过投票函数获取最终结果并返回
        ans = most_frequent_char(res,res1,res2,res3,res4)
       
        
        
        answer_list.append(ans)
     
    return answer_list

In [6]:
def write_list_to_file(file_path, data_list):
    with open(file_path, 'w') as file:
        print(f"正在写入到{file_path}文件...")
        for item in tqdm(data_list):
            file.write(f"{item}\n")

# 假设这是你的列表


# 文件路径
# file_path = './answer/answer.txt'

# 写入文件
# write_list_to_file(file_path, answer_list)

In [7]:
##写入到submit.jsonl中

def process_json(json_file_path, answer_list, output_file_path):
    # 读取 JSON 文件
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # 初始化输出数据结构
    output_data = []

    # 初始化 round_id
    round_id = -1
    current_instruction = ""

    # 处理每一项数据
    for item in data:
        # 如果 instruction 发生变化，增加 round_id
        if current_instruction != item['instruction']:
            round_id += 1
            current_instruction = item['instruction']
            current_item = {
                'id': f'round1_test_data_{round_id:03d}',
                'questions': []
            }
            output_data.append(current_item)
        else:
            current_item = output_data[-1]

        # 解析 input 字段中的选择题编号
        question_number = int(item['input'].split('选择题 ')[1].split(':')[0])

        # 获取对应答案
        answer = answer_list.pop(0)  # 使用并移除第一个答案

        # 添加问题及答案到 questions 列表
        current_item['questions'].append({'answer': answer})

    # 写入 JSONL 文件
    with open(output_file_path, 'w', encoding='utf-8') as jsonl_file:
        for item in output_data:
            jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')
    print("写入完成！！！")
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
    print('执行结束！')

In [8]:
def run_infer(MODEL_NAME,input_file_path,file_path,output_file_path):

    prompts = read_and_process_json_file(input_file_path)
    print("提示词获取成功")
    answer_list = get_answer(prompts,MODEL_NAME)
    print(len(answer_list))
    # 写入txt文件
    write_list_to_file(file_path, answer_list)
    process_json(input_file_path, answer_list, output_file_path)
    

In [9]:
# MODEL_NAME = '01ai'
MODEL_NAME = 'Qwen2-7B-Instruct-lora'
input_file_path = './dataset/output_test.json'
output_file_path = './answer/vllm_submit.jsonl'
file_path = './answer/answer.txt'
run_infer(MODEL_NAME,input_file_path,file_path,output_file_path)

提示词获取成功


Submitting tasks: 100%|██████████| 1328/1328 [3:15:47<00:00,  8.85s/it]  


1328
正在写入到./answer/answer.txt文件...


100%|██████████| 1328/1328 [00:00<00:00, 1769388.73it/s]


写入完成！！！
2024-07-29 20:01:15
执行结束！
